# group project


### -Title

### -Introduction

#### Background

#### question

#### data description

In [4]:
library(tidyverse)

raw_data <- read_csv("players.csv")

experience_levels <- c("Beginner" = 0, "Amateur" = 1, "Regular" = 2, "Veteran" = 3, "Pro" = 4)

data <- raw_data |> 
    select(experience, subscribe, played_hours) |>
    mutate(experience = as.numeric(factor(experience, levels = names(experience_levels), labels = experience_levels))) |>
    mutate(subscribe = as_factor(subscribe))


set.seed(2025)
split <- initial_split(data, prop = 0.75, strata = subscribe)
train_data <- training(split)
test_data <- testing(split)

cv_folds <- vfold_cv(train_data, v = 5, strata = subscribe)

knn_spec <- nearest_neighbor(neighbors = tune()) |> 
  set_engine("kknn") |> 
  set_mode("classification")

knn_recipe <- recipe(subscribe ~ experience + played_hours, data = train_data) |> 
  step_scale(all_predictors())

knn_workflow <- workflow() |> 
  add_recipe(knn_recipe) |> 
  add_model(knn_spec)

knn_grid <- tibble(neighbors = seq(1, 30, by = 1))

knn_results <- tune_grid(
  knn_workflow,
  resamples = cv_folds,
  grid = knn_grid,
  metrics = metric_set(accuracy))

best_k <- knn_results |> select_best("accuracy")
best_k




final_knn_spec <- nearest_neighbor(neighbors = best_k$neighbors) |> 
            set_engine("kknn") |> 
            set_mode("classification")

final_knn_workflow <- knn_workflow |> 
    finalize_workflow(best_k) |> 
    fit(data = train_data)

final_predictions <- predict(final_knn_workflow, new_data = test_data) |> 
    bind_cols(test_data)

conf_matrix <- conf_mat(final_predictions, truth = subscribe, estimate = .pred_class)
conf_matrix



accuracy_score <- accuracy(final_predictions, truth = subscribe, estimate = .pred_class)
accuracy_score

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in initial_split(data, prop = 0.75, strata = subscribe): could not find function "initial_split"


### -Methods & Results

### -Discussion